<a href="https://colab.research.google.com/github/Adah-K/ML/blob/main/Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
%matplotlib inline
from google.colab import files
import io
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report,confusion_matrix,  mean_squared_error
from sklearn.model_selection import train_test_split


## Get the Dataset

In [ ]:
dataset = files.upload()

Saving Housing Pricing.csv to Housing Pricing (1).csv


In [ ]:
data=pd.read_csv(io.BytesIO(dataset['Housing Pricing.csv']))

In [ ]:
data.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500


In [ ]:
data.shape

(1460, 77)

In [ ]:
data.drop('Id',inplace=True, axis=1)

In [ ]:
data.head(1)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500


In [ ]:
original_features=list(data.columns)
len(original_features)

76

In [ ]:
features_missing_values=list(data.columns[data.isna().any()])
len(features_missing_values)

15

In [ ]:
features_missing_values

['LotFrontage',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [ ]:
len(data)

1460

## Fill in Missing Features

In [ ]:
bfill,mean_fill = [],[]

In [ ]:
for feature in features_missing_values:
  print(feature,' ',data[feature].isna().sum()/len(data)*100,' ',data[feature].dtype)
  if data[feature].dtype=='float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

LotFrontage   17.73972602739726   float64
MasVnrType   0.547945205479452   object
MasVnrArea   0.547945205479452   float64
BsmtQual   2.5342465753424657   object
BsmtCond   2.5342465753424657   object
BsmtExposure   2.6027397260273974   object
BsmtFinType1   2.5342465753424657   object
BsmtFinType2   2.6027397260273974   object
Electrical   0.0684931506849315   object
FireplaceQu   47.26027397260274   object
GarageType   5.5479452054794525   object
GarageYrBlt   5.5479452054794525   float64
GarageFinish   5.5479452054794525   object
GarageQual   5.5479452054794525   object
GarageCond   5.5479452054794525   object


In [ ]:
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [ ]:
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace=True)

In [ ]:
for feature in bfill:
  data[feature].fillna(method='bfill',inplace=True)

In [ ]:
data.columns[data.isna().any()]

Index(['FireplaceQu'], dtype='object')

In [ ]:
data['FireplaceQu'].isna().sum()

2

In [ ]:
data['FireplaceQu'].fillna(method='ffill',inplace=True)

# Encode the Dataset

In [ ]:
categorical_features=list(data.select_dtypes(include=['object']).columns)
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [ ]:
numerical=list(set(original_features)-set(categorical_features))
numerical

['PoolArea',
 'TotRmsAbvGrd',
 'LotArea',
 '3SsnPorch',
 'SalePrice',
 'WoodDeckSF',
 'TotalBsmtSF',
 'GrLivArea',
 'HalfBath',
 '1stFlrSF',
 'Fireplaces',
 'MSSubClass',
 'OverallCond',
 'MasVnrArea',
 'BsmtFinSF1',
 'YearBuilt',
 'GarageYrBlt',
 'LotFrontage',
 'OverallQual',
 'BedroomAbvGr',
 'BsmtUnfSF',
 'BsmtFullBath',
 'KitchenAbvGr',
 'YrSold',
 'FullBath',
 'BsmtFinSF2',
 'GarageArea',
 'ScreenPorch',
 'MiscVal',
 'OpenPorchSF',
 'LowQualFinSF',
 '2ndFlrSF',
 'BsmtHalfBath',
 'GarageCars',
 'EnclosedPorch',
 'MoSold',
 'YearRemodAdd']

In [ ]:
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [ ]:
nominal=['MSZoning','LandContour','Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))

In [ ]:
target=['SalePrice']

In [ ]:
df_nominal=pd.get_dummies(data[nominal])

In [ ]:
for feature in ordinal:
  data[feature]=data[feature].astype('category').cat.codes

df_ordinal=data[ordinal]

In [ ]:
data[numerical]

,PoolArea,TotRmsAbvGrd,LotArea,3SsnPorch,SalePrice,WoodDeckSF,TotalBsmtSF,GrLivArea,HalfBath,1stFlrSF,Fireplaces,MSSubClass,OverallCond,MasVnrArea,BsmtFinSF1,YearBuilt,GarageYrBlt,LotFrontage,OverallQual,BedroomAbvGr,BsmtUnfSF,BsmtFullBath,KitchenAbvGr,YrSold,FullBath,BsmtFinSF2,GarageArea,ScreenPorch,MiscVal,OpenPorchSF,LowQualFinSF,2ndFlrSF,BsmtHalfBath,GarageCars,EnclosedPorch,MoSold,YearRemodAdd
0,0,8,8450,0,208500,0,856,1710,1,856,0,60,5,196.0,706,2003,2003.0,65.0,7,3,150,1,1,2008,2,0,548,0,0,61,0,854,0,2,0,2,2003
1,0,6,9600,0,181500,298,1262,1262,0,1262,1,20,8,0.0,978,1976,1976.0,80.0,6,3,284,0,1,2007,2,0,460,0,0,0,0,0,1,2,0,5,1976
2,0,6,11250,0,223500,0,920,1786,1,920,1,60,5,162.0,486,2001,2001.0,68.0,7,3,434,1,1,2008,2,0,608,0,0,42,0,866,0,2,0,9,2002
3,0,7,9550,0,140000,0,756,1717,0,961,1,70,5,0.0,216,1915,1998.0,60.0,7,3,540,1,1,2006,1,0,642,0,0,35,0,756,0,3,272,2,1970
4,0,9,14260,0,250000,192,1145,2198,1,1145,1,60,5,350.0,655,2000,2000.0,84.0,8,4,490,1,1,2008,2,0,836,0,0,84,0,1053,0,3,0,12,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,7,7917,0,175000,0,953,1647,1,953,1,60,5,0.0,0,1999,1999.0,62.0,6,3,953,0,1,2007,2,0,460,0,0,40,0,694,0,2,0,8,2000
1456,0,7,13175,0,210000,349,1542,2073,0,2073,2,20,6,119.0,790,1978,1978.0,85.0,6,3,589,1,1,2010,2,163,500,0,0,0,0,0,0,2,0,2,1988
1457,0,9,9042,0,266500,0,1152,2340,0,1188,2,70,9,0.0,275,1941,1941.0,66.0,7,4,877,0,1,2010,2,0,252,0,2500,60,0,1152,0,1,0,5,2006
1458,0,5,9717,0,142125,366,1078,1078,0,1078,0,20,6,0.0,49,1950,1950.0,68.0,5,2,0,1,1,2010,1,1029,240,0,0,0,0,0,0,1,112,4,1996


In [ ]:
new_data=pd.concat([df_nominal,df_ordinal,data[numerical]],axis=1)

In [ ]:
new_data.shape

(1460, 107)

# Standardizing the dataset

In [ ]:
new_data.drop('SalePrice',axis=1,inplace=True)

In [ ]:
X=new_data.to_numpy()

In [ ]:
X

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.000e+00,
        2.003e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 5.000e+00,
        1.976e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 9.000e+00,
        2.002e+03],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 5.000e+00,
        2.006e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.120e+02, 4.000e+00,
        1.996e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 6.000e+00,
        1.965e+03]])

In [ ]:
X=StandardScaler().fit_transform(X)

In [ ]:
X[0]

array([-0.08304548, -0.21585871, -0.10526316,  0.51813339, -0.41895507,
       -0.21235968, -0.18831089, -0.15899968,  0.33712564, -0.10854037,
       -0.03703704, -0.10526316, -0.20339487, -0.1398323 ,  2.95522137,
       -0.19025216, -0.27116307, -0.23917551, -0.16124951, -0.10854037,
       -0.1863522 , -0.42683279, -0.07875671, -0.22941573, -0.16998114,
       -0.23595776, -0.28963792, -0.13199092, -0.23106504, -0.20521398,
       -0.25018188, -0.13199092, -0.16347148, -0.08712888, -0.29055229,
        1.02668924, -1.23804229, -0.23012211, -0.40936914, -0.03628912,
        0.30941909,  0.83559903, -0.71775144,  0.25597761,  0.06423821,
        1.11993319,  0.31386709, -0.77797579,  0.2085023 , -0.88465767,
       -0.22571613, -0.40779461,  0.36420746,  0.303692  ,  0.28780036,
       -0.12304604, -0.03174026, -0.12579688, -0.49151573,  0.75073056,
       -0.02618016,  0.75162623,  0.22403433,  0.18797343,  0.26381257,
        0.28974476,  0.60466978, -0.41169079,  0.64046232,  0.74

In [ ]:
y=data[target]

In [ ]:
y=y.to_numpy()

In [ ]:
y.shape

(1460, 1)

In [ ]:
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [ ]:
y=StandardScaler().fit_transform(y)

In [ ]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [ ]:
X.shape

(1460, 106)

In [ ]:
y=data[target]
y

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


# Feature Selection Using L1

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
reg = LogisticRegression()
reg.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print('Learnt coefficients:', reg.coef_)
print('Learnt intercept:', reg.intercept_)


Learnt coefficients: [[ 0.40909092 -0.00500168 -0.00211977 ...  0.0567903   0.09503974
  -0.0570431 ]
 [ 0.44630241 -0.00378979 -0.00115256 ... -0.08807278  0.09101163
  -0.05039407]
 [-0.00620263 -0.00602428 -0.00320276 ... -0.0910539  -0.08259187
  -0.14145639]
 ...
 [ 0.00061782 -0.01803922  0.00065084 ... -0.00258738 -0.31549937
   0.03393748]
 [ 0.00099893 -0.01091051  0.00054873 ... -0.02689174  0.02510731
  -0.01172272]
 [ 0.00071187 -0.00569178  0.00076296 ... -0.00968722 -0.1100359
  -0.00221093]]
Learnt intercept: [-5.73205794e-01 -6.63325884e-01 -3.60248256e-01 -3.49236386e-01
 -5.47779027e-01 -2.75178314e-01 -1.42859059e-01 -6.73227747e-01
 -4.53739274e-01 -1.96848101e-01 -4.40003640e-01 -1.85734335e-01
 -2.60874262e-01 -1.60852132e-01 -4.76229224e-01 -1.86279131e-01
 -2.25350803e-01 -4.99268601e-01 -7.88569595e-01 -5.30426536e-01
  1.35785816e-01 -3.98389250e-01 -4.46954069e-01  2.43440622e-01
 -6.46850004e-01 -3.16312069e-01  1.55350842e-01 -3.40255615e-01
 -1.17824035e-0

In [ ]:
y_hat=reg.predict(X_test)

In [ ]:
len(y_hat)

438

In [ ]:
len(y_test)

438

In [ ]:
mean_squared_error(y_test,y_hat, squared=False)

58417.89902062568